In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()

target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [14]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state=42)

In [15]:
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([1.53241849, 1.59028721, 1.37850571, 1.47503591, 0.89031267]), 'score_time': array([0.11004806, 0.0687995 , 0.14365125, 0.15240932, 0.08511782]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}
0.9973541965122431 0.8905151032797809


In [16]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [17]:
rf.score(test_input, test_target)

0.8892307692307693

In [18]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [21]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)
wrong_count = 0
for i in range(len(test_input)):
    predict = et.predict(test_input[i:i+1])[0]
    answer = test_target[i:i+1][0]
    if predict != answer:
        print(test_input[i:i+1])
        wrong_count += 1
print(len(test_input), '-', wrong_count)

In [28]:
(1300-148)/1300

0.8861538461538462

In [29]:
et.score(test_input, test_target)

0.8861538461538462

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [32]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9215896435285418 0.8749283704745687


In [34]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(
    hgb, train_input, train_target, return_train_score=True
)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [35]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [38]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [50]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
lgb.fit(train_input, train_target)
print('final score', lgb.score(test_input, test_target))
print( lgb.feature_importances_)

0.935828414851749 0.8801251203079884
[LightGBM] [Info] Number of positive: 3939, number of negative: 1258
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000322 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 5197, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757937 -> initscore=1.141404
[LightGBM] [Info] Start training from score 1.141404
final score 0.8730769230769231
[ 909 1011 1080]


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.5],
    'num_leaves': [20, 31, 40, 50],
    'boosting_type' : ['gbdt', 'dart', 'goss', 'rf'],
    'random_state' : [42],
    'colsample_bytree' : [0.5, 0.7, 0.8, 1.0],
    }


grid = GridSearchCV(lgb, param_grid, cv=5, n_jobs=-1)
grid.fit(train_input, train_target)

# Print the best parameters found
print(grid.best_params_)

In [44]:
grid.score(test_input, test_target)

0.8715384615384615

In [47]:
searched_model = grid.best_estimator_
searched_model.fit(train_input, train_target)
searched_model.score(test_input, test_target)

[LightGBM] [Info] Number of positive: 3939, number of negative: 1258
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000339 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 5197, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757937 -> initscore=1.141404
[LightGBM] [Info] Start training from score 1.141404


0.8715384615384615

In [48]:
print(grid.best_params_)

{'boosting_type': 'gbdt', 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'num_leaves': 40, 'random_state': 42}
